# 🔬 Laboratório: Comparação de Modelos de Embeddings do Ollama

## Objetivo do Notebook

Neste laboratório, você aprenderá a **comparar diferentes modelos de embeddings** disponíveis via Ollama. O foco é entender:

- **O que são embeddings** e por que são fundamentais para busca semântica
- **Como diferentes modelos** (all-minilm, nomic-embed-text, mxbai-embed-large) produzem resultados distintos
- **Impacto da dimensionalidade** dos vetores no desempenho e precisão
- **Como interpretar scores de similaridade** e escolher o modelo adequado

### Por que comparar modelos?

Cada modelo de embedding tem características únicas:
- **Dimensionalidade**: modelos com mais dimensões podem capturar nuances, mas consomem mais recursos
- **Velocidade**: modelos menores são mais rápidos
- **Qualidade semântica**: alguns modelos entendem melhor contextos específicos (técnico, casual, multilíngue)

💡 **Exemplo prático**: Se você está construindo um chatbot técnico, precisa saber qual modelo melhor entende jargões da sua área.

## Pré-requisitos e Ferramentas

Para realizar as comparações, utilizaremos:

1. **LangChain**: Framework que facilita a integração com diferentes modelos de embeddings
2. **Ollama**: Plataforma para rodar modelos localmente (sem custos de API externa)
3. **FAISS**: Biblioteca de busca vetorial eficiente (criada pelo Facebook/Meta)

### Por que Ollama?

✅ **Gratuito** - Não há custos por requisição  
✅ **Privacidade** - Seus dados não saem do seu computador  
✅ **Variedade** - Diversos modelos disponíveis para teste  
✅ **Rapidez** - Latência baixa (sem latência de rede para APIs externas)

## 📊 Dataset de Teste

Vamos usar documentos de **três categorias distintas** para testar a capacidade dos modelos de distinguir contextos:

- **Tecnologia** (hardware, gadgets)
- **Culinária** (receitas, técnicas)
- **Esportes** (futebol, competições)

**Por que isso importa?** Modelos de qualidade devem conseguir identificar que "iPhone" e "RTX 4090" estão mais próximos semanticamente entre si do que com "bolo" ou "gol".

In [1]:
%pip install faiss-cpu>=1.13.1 langchain-openai==1.1.0  langchain-ollama==1.0.0 ollama==0.6.1 langchain-community>=0.4.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

# Se preferir usar Ollama Embeddings
from langchain_ollama import OllamaEmbeddings


In [3]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.


In [4]:
# Configuração do Ollama (ajuste conforme necessário)
# - Dentro do Docker: http://ollama:11434
# - Fora do Docker: http://localhost:11434
OLLAMA_API_URL = os.getenv('OLLAMA_API_URL', 'http://localhost:11434')

# Verificar se Ollama está online
def check_ollama_health() -> bool:
    try:
        response = requests.get(f'{OLLAMA_API_URL}/api/tags', timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            print(f'✅ Ollama está online! Modelos disponíveis: {len(models)}')
            for model in models:
                name = model.get('name', 'unknown')
                size = model.get('size', 0) / (1024**3)  # Convert to GB
                print(f'   - {name} ({size:.2f} GB)')
            return True
        else:
            print(f'❌ Ollama retornou status {response.status_code}')
            return False
    except Exception as e:
        print(f'❌ Erro ao conectar com Ollama: {e}')
        return False

# Testar conexão
check_ollama_health()

✅ Ollama está online! Modelos disponíveis: 3
   - all-minilm:latest (0.04 GB)
   - mxbai-embed-large:latest (0.62 GB)
   - nomic-embed-text:latest (0.26 GB)


True

## Nossos "Documentos" (Textos crus)

Note que são tópicos bem diferentes.

In [5]:

meus_textos = [
    "O novo iPhone 15 tem uma lente periscópica incrível.",    # Tecnologia
    "Para fazer um bolo macio, bata as claras em neve.",       # Culinária
    "O atacante chutou a bola no ângulo e foi gol.",           # Esporte
    "A placa de vídeo RTX 4090 roda jogos em 4K.",             # Tecnologia
    "Receita de lasanha à bolonhesa com muito queijo."         # Culinária
]

## 🧪 Comparação de Modelos do Ollama

Agora vem a parte central do laboratório! Vamos testar **3 modelos populares** e comparar seus resultados.

### 📖 Conceitos Fundamentais

#### O que é um Embedding?

Um **embedding** é uma representação numérica (vetor) de um texto. Por exemplo:

```
Texto: "gato"
Embedding: [0.12, -0.34, 0.56, ..., 0.78]  ← vetor com 768 números
```

**Por que isso é útil?** Textos semanticamente similares geram vetores próximos no espaço matemático.

**Exemplo intuitivo:**
- `"gato"` e `"felino"` → vetores próximos (conceitos similares)
- `"gato"` e `"carro"` → vetores distantes (conceitos diferentes)

#### O que é Dimensionalidade?

É o **número de componentes** do vetor:

- **all-minilm**: 384 dimensões (rápido, leve)
- **nomic-embed-text**: 768 dimensões (balanceado)
- **mxbai-embed-large**: 1024 dimensões (mais rico, mais lento)

**Analogia**: Pense em dimensões como "características" que descrevem o texto:
- 384 dimensões = descrever uma pessoa com 384 adjetivos
- 1024 dimensões = descrever com 1024 adjetivos (mais detalhado, mas mais pesado)

#### Como funciona a Busca?

1. **Indexação**: Todos os textos viram vetores e são armazenados no FAISS
2. **Query**: Sua pergunta também vira um vetor
3. **Comparação**: FAISS calcula a distância entre o vetor da pergunta e todos os vetores armazenados
4. **Ranking**: Retorna os k documentos mais próximos (menor distância = mais similar)

**Métricas de similaridade:**
- **Distância Euclidiana**: distância "em linha reta" entre vetores
- **Similaridade Cosseno**: ângulo entre vetores (usado pelo FAISS por padrão)

**Score:** Quanto **menor** o score, **mais similar** o documento (pense em "distância")

### 🎯 O que vamos testar?

Faremos a **mesma pergunta** para os 3 modelos e compararemos:
- Quais documentos cada modelo considera mais relevantes
- Os scores de similaridade
- A dimensionalidade dos embeddings

In [6]:
# Funções para comparar modelos Ollama (langchain_ollama x langchain_community)
from typing import List, Dict, Any
from IPython.display import display, Markdown

# Tentamos importar ambas as implementações (preferir langchain_ollama)
try:
    from langchain_ollama import OllamaEmbeddings as LC_OllamaEmbeddings
except Exception:
    LC_OllamaEmbeddings = None

try:
    from langchain_community.embeddings import OllamaEmbeddings as CommunityOllamaEmbeddings
except Exception:
    CommunityOllamaEmbeddings = None

def build_embeddings_for_model(model_name: str, base_url: str = OLLAMA_API_URL):
    """Tenta criar uma instância de embeddings para o modelo especificado.
    Retorna (embeddings_instance, implementation_name).
    """
    client_kwargs = {"headers": {"Authorization": f"Bearer {os.getenv('OLLAMA_API_KEY')}"}} if os.getenv('OLLAMA_API_KEY') else {}
    # 1) Preferir langchain_ollama
    if LC_OllamaEmbeddings is not None:
        try:
            emb = LC_OllamaEmbeddings(
                model=model_name,
                base_url=base_url,
                client_kwargs=client_kwargs,
                validate_model_on_init=False,
                keep_alive=5 * 60,
            )
            return emb, 'langchain_ollama'
        except Exception as e:
            last_err = e
    else:
        last_err = None
    # 2) Tentar community implementation
    if CommunityOllamaEmbeddings is not None:
        try:
            emb = CommunityOllamaEmbeddings(
                model=model_name,
                base_url=base_url,
                embed_instruction="",
                query_instruction="",
                headers=client_kwargs.get("headers") if client_kwargs else None,
                show_progress=False,
            )
            return emb, 'langchain_community'
        except Exception as e:
            last_err = e
    raise Exception(f'Não foi possível instanciar embeddings para {model_name}. Erro: {last_err}')

def compare_models(models: List[str], texts: List[str], query: str, k: int = 2, base_url: str = OLLAMA_API_URL) -> List[Dict[str, Any]]:
    """Compara uma lista de modelos: para cada modelo, instancia embeddings, constrói um FAISS index e executa uma busca.
    Retorna lista de dicionários com model, implementacao, dim e hits (texto + score) ou error.
    """
    report = []
    for m in models:
        try:
            emb, impl = build_embeddings_for_model(m, base_url=base_url)
            # conferindo a dimensão do embedding (usa embed_query como exemplo)
            emb_example = emb.embed_query("teste de dimensão")
            dim = len(emb_example)
            # construir índice FAISS e buscar
            vstore = FAISS.from_texts(texts, emb)
            hits = vstore.similarity_search_with_score(query, k=k)
            formatted_hits = [{"text": doc.page_content, "score": score} for doc, score in hits]
            report.append({"model": m, "impl": impl, "dim": dim, "hits": formatted_hits})
        except Exception as e:
            report.append({"model": m, "error": str(e)})
    return report

### 🔧 Funções Auxiliares

As funções abaixo automatizam o processo de comparação:

- **`build_embeddings_for_model()`**: Tenta instanciar um modelo de embeddings
  - Prioriza `langchain_ollama` (mais recente)
  - Faz fallback para `langchain_community` se necessário
  
- **`compare_models()`**: Para cada modelo:
  1. Cria uma instância de embeddings
  2. Verifica a dimensionalidade (com um embed de teste)
  3. Constrói um índice FAISS com os textos
  4. Executa a busca e retorna os top-k resultados

**Por que duas implementações?** O LangChain está em transição: `langchain_ollama` é o pacote moderno, mas `langchain_community` ainda é amplamente usado. Nossa função suporta ambos para máxima compatibilidade.

### 🔍 Definindo a Query de Teste

Vamos usar uma pergunta que **não contém palavras-chave exatas** dos documentos. Isso força os modelos a entender o **significado semântico**.

**Pergunta:** "Quero sugestões de hardware para computador ou celular"

**Desafio para o modelo:**
- A palavra "iPhone" não aparece na pergunta
- A palavra "RTX 4090" não aparece na pergunta
- Mas ambos são **hardware** para **celular** e **computador**

Um modelo de qualidade deve:
✅ Retornar os documentos sobre iPhone e RTX 4090  
❌ NÃO retornar documentos sobre receitas ou esportes

In [7]:
pergunta = "Quero sugestões de hardware para computador ou celular"
print(f"Query de teste: '{pergunta}'")

Query de teste: 'Quero sugestões de hardware para computador ou celular'


In [8]:
# Executar comparação entre alguns modelos comuns do Ollama
models_to_compare = ["all-minilm", "nomic-embed-text", "mxbai-embed-large"]

print('='*60)
print('🚀 INICIANDO COMPARAÇÃO DE MODELOS')
print('='*60)
print(f'Modelos a comparar: {", ".join(models_to_compare)}')
print(f'Query: "{pergunta}"')
print(f'Documentos no índice: {len(meus_textos)}')
print(f'Top-K resultados por modelo: 2')
print('='*60 + '\n')

comp_results = compare_models(models_to_compare, meus_textos, pergunta, k=2)

# Exibir resultados de forma legível
for res in comp_results:
    if 'error' in res:
        display(Markdown(f"### ❌ {res['model']}"))
        display(Markdown(f"**Erro:** {res['error']}"))
        display(Markdown("---"))
        continue
    
    display(Markdown(f"### ✅ {res['model']}"))
    display(Markdown(f"**Implementação:** `{res['impl']}`  |  **Dimensões:** `{res['dim']}`"))
    display(Markdown("**Resultados:**"))
    
    for i, h in enumerate(res['hits'], 1):
        score_emoji = "🥇" if i == 1 else "🥈"
        display(Markdown(f"{score_emoji} **Score: {h['score']:.4f}** → *{h['text']}*"))
    
    display(Markdown("---"))

🚀 INICIANDO COMPARAÇÃO DE MODELOS
Modelos a comparar: all-minilm, nomic-embed-text, mxbai-embed-large
Query: "Quero sugestões de hardware para computador ou celular"
Documentos no índice: 5
Top-K resultados por modelo: 2



### ✅ all-minilm

**Implementação:** `langchain_ollama`  |  **Dimensões:** `384`

**Resultados:**

🥇 **Score: 1.2114** → *A placa de vídeo RTX 4090 roda jogos em 4K.*

...

**Implementação:** `langchain_ollama`  |  **Dimensões:** `1024`

**Resultados:**

🥇 **Score: 0.7448** → *A placa de vídeo RTX 4090 roda jogos em 4K.*

🥈 **Score: 0.8314** → *O novo iPhone 15 tem uma lente periscópica incrível.*

---

## 📊 Como Interpretar os Resultados

### Análise dos Scores

**O que significa o score?**
- É a **distância** entre o vetor da query e o vetor do documento
- **Menor score = maior similaridade**
- Scores não têm unidade fixa (dependem da métrica usada)

**Exemplo de interpretação real dos resultados acima:**

```text
Query: "Quero sugestões de hardware para computador ou celular"

✅ all-minilm (384 dims)
🥇 Score: 1.2114 → RTX 4090
🥈 Score: 1.4282 → Lasanha

✅ nomic-embed-text (768 dims)
🥇 Score: 0.7295 → Bolo
🥈 Score: 0.7368 → Gol

✅ mxbai-embed-large (1024 dims)
🥇 Score: 0.7448 → RTX 4090
🥈 Score: 0.8314 → iPhone 15
```



### Análise Crítica dos Resultados

**🎯 Modelo `mxbai-embed-large` - O MELHOR**
- ✅ **Top-1:** RTX 4090 (score: 0.7448) - CORRETO! Hardware para computador
- ✅ **Top-2:** iPhone 15 (score: 0.8314) - CORRETO! Hardware para celular
- **Conclusão:** Entendeu perfeitamente a query, retornando exatamente os 2 documentos de hardware

**⚠️ Modelo `all-minilm` - PARCIAL**
- ✅ **Top-1:** RTX 4090 (score: 1.2114) - CORRETO! 
- ❌ **Top-2:** Lasanha (score: 1.4282) - INCORRETO! Não é hardware
- **Conclusão:** Acertou o primeiro resultado, mas trouxe um documento irrelevante em segundo lugar

**❌ Modelo `nomic-embed-text` - FALHOU**
- ❌ **Top-1:** Bolo (score: 0.7295) - INCORRETO! Receita, não hardware
- ❌ **Top-2:** Gol (score: 0.7368) - INCORRETO! Esporte, não hardware
- **Conclusão:** Não entendeu a query, retornou documentos completamente irrelevantes



### 🔍 Por que esses resultados?

**Observações importantes:**

1. **Scores não são comparáveis entre modelos diferentes**
   - `all-minilm` tem scores maiores (~1.2) 
   - `nomic-embed-text` e `mxbai-embed-large` têm scores menores (~0.7-0.8)
   - Isso é normal! Cada modelo usa um espaço vetorial diferente

2. **Mais dimensões → Melhor compreensão semântica?**
   - ✅ `mxbai-embed-large` (1024 dims): 2/2 acertos
   - ⚠️ `all-minilm` (384 dims): 1/2 acertos
   - ❌ `nomic-embed-text` (768 dims): 0/2 acertos
   - **Conclusão:** Neste caso, o modelo maior teve melhor desempenho, mas dimensionalidade sozinha não garante qualidade!

3. **`nomic-embed-text` falhou inesperadamente**
   - É um modelo popular e geralmente bom
   - Pode ter sido treinado em dados diferentes
   - Pode ter interpretação diferente de "hardware" e "sugestões"
   - **Lição:** Sempre teste com SEU domínio específico!

### 📊 Tabela Comparativa Real

| Modelo | Dims | Top-1 (Score) | Top-2 (Score) | Acertos | Qualidade |
|--------|------|---------------|---------------|---------|-----------|
| **mxbai-embed-large** | 1024 | RTX 4090 (0.74) | iPhone 15 (0.83) | ✅✅ 2/2 | 🏆 Excelente |
| **all-minilm** | 384 | RTX 4090 (1.21) | ❌ Lasanha (1.42) | ✅ 1/2 | ⚠️ Parcial |
| **nomic-embed-text** | 768 | ❌ Bolo (0.73) | ❌ Gol (0.74) | ❌ 0/2 | 💥 Falhou |

### 🎯 Conclusão desta Comparação

**Para esta query específica e este dataset:**

🥇 **Vencedor: `mxbai-embed-large`**
- Única compreensão correta da query
- Trouxe exatamente os documentos relevantes
- Ordem dos resultados faz sentido (RTX mais próximo que iPhone)

🥈 **Segundo lugar: `all-minilm`**
- Acertou o documento mais relevante
- Rápido e leve (384 dims)
- Mas trouxe um falso positivo

🥉 **Terceiro lugar: `nomic-embed-text`**
- Falhou completamente nesta query
- Pode ter boa performance em outros domínios
- Não recomendado para este tipo de busca

### 💡 Lições Aprendidas

1. **Não confie cegamente em benchmarks públicos** - teste com seus dados!
2. **Modelo maior NEM SEMPRE é melhor** - mas neste caso foi
3. **Um modelo popular pode falhar** - como o `nomic-embed-text` falhou aqui
4. **Contexto importa** - estes modelos podem ter sido treinados em domínios diferentes

## 🧠 Exercícios Práticos

### Exercício 1: Testando Outras Queries

Modifique a variável `pergunta` e reexecute a comparação. Tente:

```python
# Teste 1: Query muito específica
pergunta = "processador Intel i9 13ª geração"

# Teste 2: Query ambígua
pergunta = "como melhorar performance"

# Teste 3: Query de outro domínio
pergunta = "receitas rápidas para o jantar"
```

**Perguntas para reflexão:**
- Os modelos concordam nos resultados?
- Algum modelo se saiu melhor em um tipo específico de query?
- Queries mais específicas geram scores menores (mais confiança)?



### Exercício 2: Expandindo o Dataset

Adicione mais documentos em `meus_textos`:

```python
meus_textos = [
    # ... documentos existentes ...
    "O SSD NVMe de 2TB tem velocidades de leitura de 7000 MB/s",
    "Memória RAM DDR5 de 32GB para multitarefas pesadas",
    "Tutorial de massa de pizza caseira com fermentação natural",
]
```

**Observe:** Como a adição de documentos similares afeta os scores?



### Exercício 3: Ajustando k (Top-K)

Altere o parâmetro `k` na função `compare_models()`:

```python
comp_results = compare_models(models_to_compare, meus_textos, pergunta, k=3)  # Top-3
```

**Análise:** 
- O 3º resultado é relevante?
- Há um "gap" grande entre o score do 2º e 3º resultados?



### Exercício 4: Testando Modelos Adicionais

Se você tem outros modelos instalados no Ollama, teste-os:

```python
models_to_compare = ["all-minilm", "nomic-embed-text", "mxbai-embed-large", "llama2"]
```

⚠️ **Atenção:** Nem todos os modelos do Ollama são embeddings! Modelos como `llama2` são LLMs (geração de texto), não embeddings. A comparação falhará graciosamente com mensagem de erro.



### 🏆 Desafio Final

**Crie um benchmark completo:**

1. Defina 5 queries diferentes (variando especificidade e domínio)
2. Execute todas contra os 3 modelos
3. Calcule métricas agregadas:
   - Média dos scores por modelo
   - Taxa de concordância (quantas vezes o top-1 coincide?)
   - Tempo de execução (use `time.time()`)

4. **Escolha o vencedor** baseado no seu caso de uso:
   - Se precisar de velocidade: `all-minilm`
   - Se precisar de balanceamento: `nomic-embed-text`
   - Se precisar de máxima qualidade: `mxbai-embed-large`

## 📚 Resumo e Próximos Passos

### O que você aprendeu

✅ **Embeddings** são representações numéricas de textos que capturam significado semântico  
✅ **Dimensionalidade** afeta qualidade, velocidade e consumo de recursos  
✅ **Modelos diferentes** podem ter interpretações distintas do mesmo texto  
✅ **Scores menores** indicam maior similaridade (menor distância vetorial)  
✅ **Comparações sistemáticas** são essenciais para escolher o modelo ideal  

### Conceitos-chave para lembrar

1. **Não existe modelo perfeito** - tudo é trade-off (velocidade vs. qualidade)
2. **Sempre teste com dados reais** do seu domínio
3. **FAISS é local e rápido** - ideal para prototipagem
4. **Ollama é gratuito** - aproveite para experimentar!


### 📖 Recursos Adicionais

- [Ollama Models](https://ollama.ai/library) - Catálogo completo de modelos
- [FAISS Documentation](https://github.com/facebookresearch/faiss/wiki) - Guias e tutoriais
- [LangChain Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) - Referência oficial
- [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) - Benchmark de embeddings

---

**🎓 Parabéns por completar o laboratório!** Agora você tem as ferramentas para tomar decisões informadas sobre modelos de embeddings.